In [ ]:
import sys
import random
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from sklearn.preprocessing import RobustScaler

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torch.nn import ModuleList, Linear, BatchNorm1d
import torch.nn.functional as F

In [ ]:
if torch.cuda.is_available():
    !pip install --no-index --no-deps /kaggle/input/torch-geometric-gpu/wheelhouse/*.whl
else:
    !pip install --no-index --no-deps /kaggle/input/torch-geometric-cpu/wheelhouse/*.whl

In [ ]:
!rm  -r software
!scp -r /kaggle/input/graphnet-and-dependencies/software .

In [ ]:
!cd software/graphnet;pip install --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

In [ ]:
import torch_cluster
import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import EdgeConv, global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.nn.pool import knn_graph
from torch_geometric.typing import Adj
from torch_geometric.utils.homophily import homophily

from torch_scatter import scatter_max, scatter_mean, scatter_min, scatter_sum


INPUT_PATH = "/kaggle/input/icecube-neutrinos-in-deep-ice"
TRANSPARENCY_PATH = "/kaggle/input/icecubetransparency/ice_transparency.txt"
MODEL_PATH = "/kaggle/input/gnn-ready-model/EdgeConvV10.pth"
MODE = 'resume'
TRAIN_BATCHES = 3

GLOBAL_POOLINGS = {
    "max": global_max_pool,
    "add": global_add_pool,
    "mean": global_mean_pool,
}

_dtype = {
    "batch_id": "int16",
    "event_id": "int64",
}

Setting seed = 0 for reproducibility

In [ ]:
def seed_settings(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_settings(seed=0)

## Dataset

### Making features using ice transparency data

In [ ]:
# Ice Data Taken from Measurement of South Pole ice transparency with the IceCube
# LED calibration system https://arxiv.org/pdf/1301.5361.pdf
!cat /kaggle/input/icecubetransparency/ice_transparency.txt

In [ ]:
def ice_transparency(data_path, center_depth=1950):
    """
    Calculates normalized ice scattering and absorption for all depths,
    using data from Measurement of South Pole ice transparency with the IceCube
    LED calibration system

    Center_depth is center of IceCube underground array
    """
    sensor_amount = 500

    df = pd.read_csv(data_path, delim_whitespace=True)
    df["z_score"] = (df["depth"] - center_depth) / sensor_amount
    df[["scattering_len_norm", "absorption_len_norm"]] = RobustScaler().fit_transform(
        df[["scattering_len", "absorption_len"]]
    )

    # These are both roughly equivalent after scaling
    # TODO: Make new feature (1-(sc+ab)/2)?
    f_scattering = interp1d(df["z_score"], df["scattering_len_norm"])
    f_absorption = interp1d(df["z_score"], df["absorption_len_norm"])
    return f_scattering, f_absorption

### Making Dataset class

In [ ]:
class IceCubeDataset(Dataset):
    """
    Used to transfer data from table format to graph format, with each event
    represented by singular graph
    """
    def __init__(
        self,
        batch_id,
        event_ids,
        sensor_df,
        mode="test",
        y=None,
        pulse_limit=220,  # >95% events have 220 pulses or less
    ):
        super().__init__()
        self.y = y
        self.event_ids = event_ids
        self.batch_df = pd.read_parquet(f"{INPUT_PATH}/{mode}/batch_{batch_id}.parquet")
        self.sensor_df = sensor_df
        self.pulse_limit = pulse_limit
        self.f_scattering, self.f_absorption = ice_transparency(TRANSPARENCY_PATH)
        # Rescaling parameters
        self.batch_df["time"] = (self.batch_df["time"] - 1.0e04) / 3.0e4
        self.batch_df["charge"] = np.log10(self.batch_df["charge"]) / 3.0
        self.batch_df["auxiliary"] = self.batch_df["auxiliary"].astype(int) - 0.5

    def len(self):
        return len(self.event_ids)

    def get(self, idx):
        """
        Returns data for given event index
        """
        event_id = self.event_ids[idx]
        event = self.batch_df.loc[event_id]

        # represent each event by a single graph
        event = pd.merge(event, self.sensor_df, on="sensor_id")
        col = ["x", "y", "z", "time", "charge", "auxiliary"]

        x = event[col].values.astype(float)
        x = torch.tensor(x, dtype=torch.float32)
        # A data object describing a homogeneous graph
        data = Data(x=x, n_pulses=torch.tensor(x.shape[0], dtype=torch.int32))

        # Downsample the large events
        if data.n_pulses > self.pulse_limit:
            data.x = data.x[np.random.choice(data.n_pulses, self.pulse_limit)]
            data.n_pulses = torch.tensor(self.pulse_limit, dtype=torch.int32)

        # Builds graph from the k-nearest neighbours.
        data.edge_index = knn_graph(
            data.x,  # x, y, z
            k=8,
            cosine=False,
            loop=False
        )

        if self.y is not None:
            y = self.y.loc[idx, :].values
            y = torch.tensor(y, dtype=torch.float32)
            data.y = y

        return data

### Preparing sensors data

In [ ]:
def prepare_sensors():
    """
    Preparing sensors position and quantum efficiency to be used
    as input to model
    """
    sensors = pd.read_csv(f"{INPUT_PATH}/sensor_geometry.csv").astype(
        {
            "sensor_id": np.int16,
            "x": np.float32,
            "y": np.float32,
            "z": np.float32,
        }
    )
    sensors["string"] = 0
    sensors["qe"] = 1

    # 60 DOMs per string
    for i in range(len(sensors) // 60):
        start, end = i * 60, (i * 60) + 60
        sensors.loc[start:end, "string"] = i

        # High Quantum Efficiency in the lower 50 DOMs
        # https://arxiv.org/pdf/2209.03042.pdf (Figure 1)
        # DeepCore located on 8 strings
        if i in range(78, 86):
            start_veto, end_veto = i * 60, (i * 60) + 10
            start_core, end_core = end_veto + 1, (i * 60) + 60
            # The DOMs deployed in DeepCore have an efficiency that is roughly
            # a factor of 1.35 times the QE of standard DOMs
            sensors.loc[start_core:end_core, "qe"] = 1.35

    # https://github.com/graphnet-team/graphnet/blob/b2bad25528652587ab0cdb7cf2335ee254cfa2db/src/graphnet/models/detector/icecube.py#L33-L41
    # Assume that "rde" (relative dom efficiency) is equivalent to QE
    sensors["x"] /= 500
    sensors["y"] /= 500
    sensors["z"] /= 500
    sensors["qe"] -= 1.25
    sensors["qe"] /= 0.25

    return sensors

In [ ]:
print(f"{INPUT_PATH}/train_meta.parquet")

In [ ]:
sensors = prepare_sensors()
sensors

In [ ]:
meta = pd.read_parquet(
   f"{INPUT_PATH}/train_meta.parquet", columns=["batch_id", "event_id", "azimuth", "zenith"]
).astype(_dtype)
meta

In [ ]:
batch_ids = meta["batch_id"].unique()

## Loss Function

In [ ]:
class DifferentiableClamp(torch.autograd.Function):
    """
    In the forward pass this operation behaves like torch.clamp.
    But in the backward pass its gradient is 1 everywhere, as if instead of clamp one had used the identity function.
    """
    @staticmethod
    def forward(ctx, x, min_val, max_val):
        return x.clamp(min_val, max_val)

    @staticmethod
    def backward(ctx, grad_output):
        # None needed because of the optional arguments min_val and max v_val
        return grad_output.clone(), None, None

class AngularLoss(pl.LightningModule):
    """
    Evaluating mean angular error between the predicted and true event origins
    """
    def __init__(self, eps = .001): #gradients explode, so have to add eps
        super().__init__()
        self.high =1-eps
        self.low = -1+eps
        self.Clamp = DifferentiableClamp()
   
    def forward(self, y_pred, y_true):
        y_true = angles_to_unit_vector(y_true[:,0], y_true[:,1])
        y_pred = angles_to_unit_vector(y_pred[:,0], y_pred[:,1])
        
        scalar_prod = torch.sum(y_pred*y_true,dim = 1)
        scalar_prod = self.Clamp.apply(scalar_prod, self.low, self.high)
        return torch.mean(torch.abs(torch.arccos(scalar_prod)))

In [ ]:
def angles_to_unit_vector(azimuth, zenith):
    """
    Transforms azimuth and zenith angles to unit vector in x,y,z coordinates
    """
    return torch.stack([
        torch.cos(azimuth) * torch.sin(zenith),
        torch.sin(azimuth) * torch.sin(zenith),
        torch.cos(zenith)
    ], dim=1)

In [ ]:
def angular_dist_score(y_pred, y_true):
    """
    Competition metric used for validation
    """
    y_pred = y_pred.cpu()
    y_true = y_true.cpu()
    az_true = y_true[:,0]
    az_pred = y_pred[:,0]
    zen_true = y_true[:,1]
    zen_pred = y_pred[:,1]
    #if not (np.all(np.isfinite(az_true)) and
    #        np.all(np.isfinite(zen_true)) and
    #        np.all(np.isfinite(az_pred)) and
    #        np.all(np.isfinite(zen_pred))):
    #    raise ValueError("All arguments must be finite")
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    return np.average(np.abs(np.arccos(scalar_prod)))

## Graph Summary

In [ ]:
batch = 1
event_ids = meta[meta["batch_id"] == batch]["event_id"].tolist()
y = meta[meta["batch_id"] == batch][['zenith', 'azimuth']].reset_index(drop=True)
train_dataset = IceCubeDataset(batch, event_ids, sensors, mode='train', y=y)

In [ ]:
print()
print(f'Dataset: {train_dataset}:')
print('====================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')

train_data = train_dataset[30]  # Get the first graph object.

print()
print(train_data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {train_data.num_nodes}')
print(f'Number of edges: {train_data.num_edges}')
print(f'Average node degree: {train_data.num_edges / train_data.num_nodes:.2f}')
print(f'Has isolated nodes: {train_data.has_isolated_nodes()}')
print(f'Has self-loops: {train_data.has_self_loops()}')
print(f'Is undirected: {train_data.is_undirected()}')

In [ ]:
import networkx as nx
from torch_geometric.utils import to_networkx

nxg = to_networkx(train_data)
nx.draw(nxg, with_labels=True)

## Model

### Layers

In [ ]:
class DynEdgeConv(EdgeConv, pl.LightningModule):
    """
    Dynamical edge convolution layer.
    """

    def __init__(self, nn, aggr = "max", nb_neighbors = 8, features_subset = None, **kwargs,):
        """
        Construct `DynEdgeConv`.
        Args:
            nn: The MLP/torch.Module to be used within the `EdgeConv`.
            aggr: Aggregation method to be used with `EdgeConv`.
            nb_neighbors: Number of neighbours to be clustered after the
                `EdgeConv` operation.
            features_subset: Subset of features in `Data.x` that should be used
                when dynamically performing the new graph clustering after the
                `EdgeConv` operation. Defaults to all features.
            **kwargs: Additional features to be passed to `EdgeConv`.
        """
        # Check(s)
        if features_subset is None:
            features_subset = slice(None)  # Use all features
        assert isinstance(features_subset, (list, slice))

        # Base class constructor
        super().__init__(nn=nn, aggr=aggr, **kwargs)

        # Additional member variables
        self.nb_neighbors = nb_neighbors
        self.features_subset = features_subset

    def forward(self, x, edge_index, batch = None):
        """
        Forward pass.
        """
        # Standard EdgeConv forward pass
        x = super().forward(x, edge_index)

        # Recompute adjacency
        edge_index = knn_graph(
            x=x[:, self.features_subset],
            k=self.nb_neighbors,
            batch=batch,
        ).to(self.device)

        return x, edge_index

### Global Variables

In [ ]:
def calculate_xyzt_homophily(x, edge_index, batch):
    """
    Calculate xyzt-homophily from a batch of graphs.

    Homophily is a graph scalar quantity that measures the likeness of
    variables in nodes. Notice that this calculator assumes a special order of
    input features in x.

    Returns:
        Tuple, each element with shape [batch_size,1].
    """
    hx = homophily(edge_index, x[:, 0], batch).reshape(-1, 1)
    hy = homophily(edge_index, x[:, 1], batch).reshape(-1, 1)
    hz = homophily(edge_index, x[:, 2], batch).reshape(-1, 1)
    ht = homophily(edge_index, x[:, 3], batch).reshape(-1, 1)
    return hx, hy, hz, ht

In [ ]:
def calculate_global_variables(x, edge_index, batch, *additional_attributes):
        """
        Calculate global variables.
        """
        # Calculate homophily (scalar variables)
        h_x, h_y, h_z, h_t = calculate_xyzt_homophily(x, edge_index, batch)

        # Calculate mean features
        global_means = scatter_mean(x, batch, dim=0)

        # Add global variables
        global_variables = torch.cat(
            [
                global_means,
                h_x,
                h_y,
                h_z,
                h_t,
            ],
            dim=1,
        )

        return global_variables

### Building model

In [ ]:
class EdgeModel(torch.nn.Module):
    def __init__(self, dim_h):
        super(EdgeModel, self).__init__()
        self.n_features = train_dataset.num_features
        self.n_outputs = 2
        self.convolution = nn.ModuleList([DynEdgeConv(nn.Sequential(Linear(2*self.n_features, 128), nn.LeakyReLU(),
                                             Linear(128, 256), nn.LeakyReLU())),
                                         DynEdgeConv(nn.Sequential(Linear(512, 336), nn.LeakyReLU(),
                                             Linear(336, 256), nn.LeakyReLU())),
                                         DynEdgeConv(nn.Sequential(Linear(512, 336), nn.LeakyReLU(),
                                             Linear(336, 256), nn.LeakyReLU()))])
        self.post_process = nn.Sequential(Linear(774, 336), nn.LeakyReLU(),
                                          Linear(336, 256), nn.LeakyReLU())
        self.readout = nn.Sequential(Linear(778, 128), nn.LeakyReLU())
        self.out = nn.Sequential(Linear(128, self.n_outputs),
                                      BatchNorm1d(self.n_outputs), nn.LeakyReLU())
        
    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        global_features = calculate_global_variables(x, 
                                                     edge_index,
                                                     batch,
                                                    )
        
        graphs = [x]
        for i, layer in enumerate(self.convolution):
            graph = layer(graphs[i], edge_index)
            graphs.append(graph)
        x = torch.cat(graphs, dim=1)
        
        x = self.post_process(x)
        
        pool_x = []
        for pool in GLOBAL_POOLINGS.values():
            pool_x.append(pool(x, batch))
        
        pool_x.append(global_features)
        pool_x = torch.cat(pool_x, dim=1)
        
        x = self.readout(pool_x)
        x = self.out(x)
        return x
    
    def fit(self, loader, batch_size=256, epochs=5, device='cuda'):
        loss_fn = AngularLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)  
        self.train()
        
        for epoch in range(epochs+1):
            epoch_loss = 0
            for batch in loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = self(batch.x, batch.edge_index, batch.batch)
                loss = loss_fn(out, batch.y.reshape(out.size()))
                epoch_loss += loss.item()
                loss.backward()
                optimizer.step()   
            epoch_loss = epoch_loss / len(loader)
            print(f'Epoch: {epoch}, Loss: {epoch_loss:.4f}')

    @torch.no_grad()
    def validate(self, data, batch_size=256, device='cuda'):
        loss_fn = angular_dist_score
        epoch_loss = 0
        predictions = []
        
        self.eval()
        for batch in loader:
            batch = batch.to(device)
            
            out = self(batch.x, batch.edge_index, batch.batch)
                
            loss = loss_fn(out, batch.y.reshape(out.size()))
                
            epoch_loss += loss.item()
            
            predictions.append(out)
                
        epoch_loss = epoch_loss / len(loader)
        print(f'Loss: {epoch_loss:.4f}')
        return torch.cat(predictions, 0)
    
    @torch.no_grad()
    def test(self, data, device='cuda'):
        loader = DataLoader(data, batch_size=128, shuffle=False)
        predictions = []
        
        self.eval()
        for data in loader:
            data.to(device)
            out = self(data.x, data.edge_index, data.batch)
            predictions.append(out)
        return torch.cat(predictions, 0)

In [ ]:
model = EdgeModel(dim_h=128)
print(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using ', device)
model = model.to(device)

## Training

In [ ]:
if MODE == 'train':
    for batch in random.sample(range(1, 600), TRAIN_BATCHES):
        i = 1
        path = '/kaggle/working/EdgeConvV3_batch_' + str(i) + '.pth'
        print(f'Batch: {batch:03d}')
        print('====================')
        event_ids = meta[meta["batch_id"] == batch]["event_id"].tolist()
        y = meta[meta["batch_id"] == batch][['azimuth', 'zenith']].reset_index(drop=True)

        train_dataset = IceCubeDataset(batch, event_ids, sensors, mode='train', y=y)
        loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
        print(device)
        model.fit(loader, device=device)
        torch.save(model, path)
elif MODE == 'resume':
    model = torch.load(MODEL_PATH)
    for batch in random.sample(range(1, 600), TRAIN_BATCHES):
        i = 1
        path = '/kaggle/working/EdgeConvV3_batch_' + str(i) + '.pth'
        print(f'Batch: {batch:03d}')
        print('====================')
        event_ids = meta[meta["batch_id"] == batch]["event_id"].tolist()
        y = meta[meta["batch_id"] == batch][['azimuth', 'zenith']].reset_index(drop=True)

        train_dataset = IceCubeDataset(batch, event_ids, sensors, mode='train', y=y)
        loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
        print(device)
        model.fit(loader, device=device)
        torch.save(model, path)
else:
    model = torch.load(MODEL_PATH)

In [ ]:
torch.save(model, '/kaggle/working/EdgeConvV6.pth')

## Validation

## TODO
validate using competition metric

In [ ]:
batch = random.randint(601, 660)
print(batch)

event_ids = meta[meta["batch_id"] == batch]["event_id"].tolist()
y = meta[meta["batch_id"] == batch][['azimuth', 'zenith']].reset_index(drop=True)

val_dataset = IceCubeDataset(batch, event_ids, sensors, mode='train', y=y)
loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
output = model.validate(loader, device=device)

In [ ]:
y

In [ ]:
sub = {
    "event_id": event_ids,
    "azimuth": output[:, 0].cpu(),# % (2 * math.pi),
    "zenith": output[:, 1].cpu(), #% math.pi,
}

sub = pd.DataFrame(sub)
sub.to_csv("submission.csv", index=False)

In [ ]:
sub

## Infering

In [ ]:
test_meta = pd.read_parquet(
    f"{INPUT_PATH}/test_meta.parquet", columns=["batch_id", "event_id"]
).astype(_dtype)
test_meta

In [ ]:
event_id_labels = []
batch_preds = []
batch_ids = test_meta["batch_id"].unique()
for batch in batch_ids:
    test_event_ids = test_meta[test_meta["batch_id"] == batch]["event_id"].tolist()
    test_dataset = IceCubeDataset(batch, test_event_ids, sensors, mode='test', y=None)
    # .cpu().detach()
    batch_preds.append(model.test(test_dataset))
    event_id_labels.extend(test_meta[test_meta["batch_id"] == batch]["event_id"].tolist())
    print("Finished batch", batch)

## Making Submission

In [ ]:
import math

output = torch.cat(batch_preds, 0).cpu()
sub = {
    "event_id": event_id_labels,
    "azimuth": output[:, 0],# % (2 * math.pi),
    "zenith": output[:, 1], #% math.pi,
}

sub = pd.DataFrame(sub)
sub.to_csv("submission.csv", index=False)

In [ ]:
sub